<a href="https://colab.research.google.com/github/mariyajoseph2002/ml/blob/main/training_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Embedding


In [ ]:
import random
# Set all random seeds for reproducibility
np.random.seed(42)        # NumPy operations
random.seed(42)           # Python's built-in random
tf.random.set_seed(42)    # TensorFlow/Keras

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving new_traindata (1).txt to new_traindata (1).txt


In [ ]:
data=pd.read_csv("new_traindata (1).txt",sep=";")
data.columns=["Text","Emotion"]
data.head(5)

,Text,Emotion
0,"Hearing a professor say, ""Many of you will fai...",fear
1,i know and i feel that its time to wake up to ...,joy
2,i am feeling that bitter sweetness that comes ...,anger
3,i wasn t feeling pressured even if this was th...,fear
4,im starting to feel and think as if i dont wan...,sadness


In [ ]:
data.describe()

,Text,Emotion
count,24799,24799
unique,14534,11
top,if I had just done one thing differently none ...,joy
freq,23,3000


In [ ]:
data['Emotion'] = data['Emotion'].str.strip()

In [ ]:
texts=data["Text"].tolist()
labels=data["Emotion"].tolist()

TypeError: Series.count() takes 1 positional argument but 2 were given

In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(texts)

In [ ]:
import pickle

# Save tokenizer after training
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)


In [ ]:
import json

tokenizer_json = tokenizer.word_index
with open("tokenizer.json", "w") as f:
    json.dump(tokenizer_json, f)
print("✅ Tokenizer saved! Total words:", len(tokenizer.word_index))

✅ Tokenizer saved! Total words: 13427


In [ ]:
from google.colab import files
files.download("tokenizer.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
sequences=tokenizer.texts_to_sequences(texts)
#sequences = [seq if seq else [0] for seq in tokenizer.texts_to_sequences(texts)]
max_length = min(66, max([len(seq) for seq in sequences]))  # Cap at 66 words
#max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences,maxlen = max_length)

In [ ]:
le=LabelEncoder()
labels=le.fit_transform(labels)

In [ ]:
one_hot_labels=keras.utils.to_categorical(labels)

In [ ]:
#x_train,x_test,y_train,y_test=train_test_split(padded_sequences,one_hot_labels,test_size=0.2)
x_train, x_test, y_train, y_test = train_test_split(
    padded_sequences, one_hot_labels, test_size=0.2, stratify=one_hot_labels
)


In [ ]:
print("Emotion to Index Mapping:", le.classes_)


Emotion to Index Mapping: ['anger' 'fear' 'guilt' 'hope' 'joy' 'loneliness' 'love' 'neutral'
 'sadness' 'surprise']


In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

In [ ]:
from keras.regularizers import l2

In [ ]:
model=Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1,output_dim=128,input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128,activation="relu", kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(units=len(one_hot_labels[0]),activation="softmax"))
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test), callbacks=[early_stopping])


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


620/620 ━━━━━━━━━━━━━━━━━━━━ 26s 39ms/step - accuracy: 0.2738 - loss: 2.2755 - val_accuracy: 0.5675 - val_loss: 1.3564
Epoch 2/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 25s 40ms/step - accuracy: 0.5651 - loss: 1.3436 - val_accuracy: 0.6845 - val_loss: 1.1063
Epoch 3/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 39s 37ms/step - accuracy: 0.7164 - loss: 1.0841 - val_accuracy: 0.7772 - val_loss: 0.9471
Epoch 4/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 23s 37ms/step - accuracy: 0.8244 - loss: 0.8713 - val_accuracy: 0.8206 - val_loss: 0.8354
Epoch 5/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 40s 36ms/step - accuracy: 0.8884 - loss: 0.7077 - val_accuracy: 0.8413 - val_loss: 0.7783
Epoch 6/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 40s 34ms/step - accuracy: 0.9204 - loss: 0.6071 - val_accuracy: 0.8550 - val_loss: 0.7263
Epoch 7/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 42s 36ms/step - accuracy: 0.9369 - loss: 0.5306 - val_accuracy: 0.8522 - val_loss: 0.7381
Epoch 8/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 42s 37ms/step - accuracy: 0.9514 - loss: 0.4913 - val_accurac

In [ ]:
def adjust_softmax(output, temperature=0.5):
    exp_preds = np.exp(np.log(output) / temperature)
    return exp_preds / np.sum(exp_preds)

In [ ]:
dummy_input = np.random.rand(1, max_length).astype(np.float32)  # Dummy input to test
raw_output = model.predict(dummy_input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step


In [ ]:
adjusted_output = adjust_softmax(raw_output)

print("\n🔢 Raw Model Output:\n", raw_output)
print("\n🔥 Adjusted Softmax Output:\n", adjusted_output)


🔢 Raw Model Output:
 [[0.06040512 0.50784284 0.0040582  0.01482723 0.1711174  0.00466959
  0.15345852 0.01941662 0.04799281 0.01621165]]

🔥 Adjusted Softmax Output:
 [[1.1489141e-02 8.1207961e-01 5.1856892e-05 6.9224468e-04 9.2199445e-02
  6.8659072e-05 7.4151844e-02 1.1871000e-03 7.2525777e-03 8.2755066e-04]]


In [ ]:
from sklearn.metrics import f1_score
# Assuming y_test and y_pred are one-hot encoded
y_test_categorical = np.argmax(y_test, axis=1)  # Convert one-hot to categorical labels
y_pred_categorical = np.argmax(y_pred, axis=1)  # Convert model predictions to categorical labels

# Compute F1 score (for multi-class)
f1 = f1_score(y_test_categorical, y_pred_categorical, average='weighted')

print("F1 Score:", f1)

In [ ]:
input_text=" I am very happy today! "
input_sequence=tokenizer.texts_to_sequences([input_text])
padded_input_sequence=pad_sequences(input_sequence,maxlen=max_length)
prediction=model.predict(padded_input_sequence)
predicted_label=le.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
['joy']


In [ ]:
prediction = model.predict(x_test, batch_size=32)  # Ensure batch size is set properly


155/155 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [ ]:
from sklearn.metrics import f1_score
import numpy as np

# Assuming y_test and y_pred are one-hot encoded
y_test_categorical = np.argmax(y_test, axis=1)  # Convert one-hot to categorical labels
y_pred_categorical = np.argmax(prediction, axis=1)  # Convert model predictions to categorical labels
print("y_test shape:", y_test.shape)
print("prediction shape:", prediction.shape)



y_test shape: (4960, 10)
prediction shape: (4960, 10)


In [ ]:
# Compute F1 score (for multi-class)
f1 = f1_score(y_test_categorical, y_pred_categorical, average='macro')

print("F1 Score:", f1)

F1 Score: 0.8667705511853623


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model_path="/content/drive/My Drive/Colab Notebooks/Data/emotio_model.h5"

In [ ]:
model.save(model_path)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

Saved artifact at '/tmp/tmpp0_8zj5x'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 66), dtype=tf.float32, name='keras_tensor_12')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  138875306708048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138875306708432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138875306709584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138875306709200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138875306710352: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
with open("emotio_model.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model saved as emotio_model.tflite")

TFLite model saved as emotio_model.tflite


In [ ]:
from google.colab import files
files.download("emotio_model.tflite")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
interpreter = tf.lite.Interpreter(model_path="emotio_model.tflite")
interpreter.allocate_tensors()

In [ ]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
# Run inference
input_data = np.array(padded_input_sequence, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

print("📌 TFLite Raw Output:", output_data)

NameError: name 'padded_input_sequence' is not defined

In [ ]:
dummy_input = np.random.rand(1, 66).astype(np.float32)  # Adjust 66 based on model input shape
output = model.predict(dummy_input)

# Print output order
print("\nModel Output Order:\n", output)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step

Model Output Order:
 [[0.06040512 0.50784284 0.0040582  0.01482723 0.1711174  0.00466959
  0.15345852 0.01941662 0.04799281 0.01621165]]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step

Model Output:
 [[0.0980273  0.13754573 0.26914057 0.34854153 0.09973297 0.04701192]]


In [ ]:
import numpy as np

# Create test sentences
test_sentences = ["I am very happy today", "I feel so sad and lonely.", "That was a scary experience.","I was scared when i so that","my teacher told me my future is bright"]

# Tokenize and pad them just like training data
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, maxlen=max_length)
# Get predictions
predictions = model.predict(test_padded)
# Apply to predictions

predicted_labels = le.inverse_transform(np.argmax(predictions, axis=1))

# Print results
# Get final predicted index

for text, pred, adj_pred in zip(test_sentences, predictions, predicted_labels):
    print(f"📝 Input: {text} \n🔮 Raw Output: {pred} \n🔥 Adjusted Output: {adj_pred}" )

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
📝 Input: I am very happy today 
🔮 Raw Output: [1.3690714e-02 2.0885953e-01 7.5056421e-05 8.4439283e-03 5.9457248e-01
 1.4137905e-04 1.5014140e-01 1.3340312e-02 1.0328417e-02 4.0675653e-04] 
🔥 Adjusted Output: joy
📝 Input: I feel so sad and lonely. 
🔮 Raw Output: [8.9138836e-02 5.7539567e-03 3.8473451e-04 4.6396309e-07 9.2215585e-03
 4.0743902e-04 1.4873405e-04 5.4181000e-05 8.9294797e-01 1.9421300e-03] 
🔥 Adjusted Output: sadness
📝 Input: That was a scary experience. 
🔮 Raw Output: [1.02279976e-01 5.30467570e-01 1.94594759e-04 2.61816522e-03
 2.80872226e-01 2.38493638e-04 4.96877916e-02 4.93114162e-03
 2.67166346e-02 1.99337234e-03] 
🔥 Adjusted Output: fear
📝 Input: I was scared when i so that 
🔮 Raw Output: [7.0076026e-02 9.2022085e-01 1.0413475e-04 1.6286783e-04 3.0499955e-03
 1.1667311e-04 3.1773832e-03 2.7422199e-04 5.3143402e-04 2.2864356e-03] 
🔥 Adjusted Output: fear
📝 Input: my teacher told me my future is bright 
🔮 Raw Output: [1.0569339e-0

In [ ]:
for sentence in test_sentences:
    print(tokenizer.texts_to_sequences([sentence]))  # See if words are properly tokenized


[[1, 28, 76, 237, 104]]
[[1, 3, 21, 413, 5, 354]]
[[8, 16, 6, 3020, 580]]
[[1, 16, 362, 32, 1, 21, 8]]
[[7, 1011, 454, 15, 7, 269, 18, 1231]]


In [ ]:
print(data["Emotion"].value_counts())  # Check if emotions are balanced


Emotion
joy           3000
sadness       3000
love          2400
surprise      2400
anger         2398
fear          2394
guilt         2300
loneliness    2300
hope          2300
neutral       2300
Name: count, dtype: int64


In [ ]:
print(tokenizer.word_index["happy"])   # Should match the ID for "happy" in tokenized output
print(tokenizer.word_index["sad"])     # Should match the ID for "sad"
print(tokenizer.word_index["scary"])   # Should match the ID for "scary"


196
307
2654


In [ ]:
print(tokenizer.word_index.get("scary"))
print(tokenizer.texts_to_sequences(["That was a scary experience."]))

2654
[[8, 14, 6, 2654, 415]]


In [ ]:
input_text="  I feel so sad and lonely. "
input_sequence=tokenizer.texts_to_sequences([input_text])
padded_input_sequence=pad_sequences(input_sequence,maxlen=max_length)
prediction=model.predict(padded_input_sequence)
predicted_label=le.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
['sadness']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model_path="/content/drive/My Drive/Colab Notebooks/Data/emotionn_model.h5"
model.save(model_path)

In [ ]:
files.download(model_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print("\nEmotion distribution:\n", data["Emotion"].value_counts())
dummy_input = np.random.rand(1, 66).astype(np.float32)  # Adjust 66 based on model input shape
output = model.predict(dummy_input)

# Print output order
print("\nModel Output Order:\n", output)


Emotion distribution:
 Emotion
joy         3000
sadness     3000
surprise    2400
love        2400
anger       2398
fear        2394
Name: count, dtype: int64
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step

Model Output Order:
 [[0.3676894  0.15944749 0.2444347  0.14175314 0.03707249 0.04960279]]


In [ ]:
print(tokenizer.texts_to_sequences(["happy", "sad", "angry"]))


[[196], [307], [263]]


In [ ]:
print("✅ First 10 words in tokenizer: ${wordToIndex.entries.take(10).toList()}");


✅ First 10 words in tokenizer: ${wordToIndex.entries.take(10).toList()}


In [ ]:
print(tokenizer.word_index.get('scared', "Not Found"))


270


In [ ]:
print(tokenizer.word_index)  # See mapping of words → indices


{'i': 1, 'feel': 2, 'and': 3, 'to': 4, 'the': 5, 'a': 6, 'of': 7, 'that': 8, 'my': 9, 'feeling': 10, 'in': 11, 'it': 12, 'like': 13, 'was': 14, 'but': 15, 'me': 16, 'for': 17, 'so': 18, 'im': 19, 'is': 20, 'have': 21, 'with': 22, 'this': 23, 'am': 24, 'about': 25, 'be': 26, 'not': 27, 'just': 28, 'as': 29, 'on': 30, 'you': 31, 'when': 32, 'at': 33, 'out': 34, 'or': 35, 'all': 36, 'because': 37, 'by': 38, 'up': 39, 'do': 40, 'more': 41, 'time': 42, 'can': 43, 'had': 44, 'really': 45, 'know': 46, 'how': 47, 'they': 48, 'would': 49, 'even': 50, 't': 51, 'are': 52, 'very': 53, 'if': 54, 'what': 55, 'been': 56, 'her': 57, 'will': 58, 'myself': 59, 'he': 60, 'little': 61, 'being': 62, 'never': 63, 'from': 64, 'get': 65, 'them': 66, 'one': 67, 'people': 68, 'still': 69, 'him': 70, 'now': 71, 'thought': 72, 'love': 73, 'want': 74, 'an': 75, 'something': 76, 'some': 77, 'who': 78, 'she': 79, 'much': 80, 'think': 81, 'we': 82, 'bit': 83, 'life': 84, 'there': 85, 'no': 86, 'could': 87, 'its': 88,

In [ ]:
import numpy as np

input_text = ["not feeling good"]
input_sequence = tokenizer.texts_to_sequences(input_text)
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)

prediction = model.predict(padded_input_sequence)
print("📌 Raw Output:", prediction)
print("🔮 Predicted Emotion:", le.inverse_transform([np.argmax(prediction[0])]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
📌 Raw Output: [[0.00456881 0.06761686 0.78635746 0.11357    0.02700583 0.00088109]]
🔮 Predicted Emotion: ['joy']


In [ ]:
import numpy as np

# Create test sentences
test_sentences = ["I am very happy today!", "I feel so sad and lonely.", "That was a scary experience.","I feel so intimidated by his actions"]

# Tokenize and pad them just like training data
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, maxlen=max_length)

# Get predictions
predictions = model.predict(test_padded)

# Convert predictions to emotion labels
predicted_labels = le.inverse_transform(np.argmax(predictions, axis=1))

# Print results
for text, label, pred in zip(test_sentences, predicted_labels, predictions):
    print(f"📝 Input: {text} \n🔮 Predicted Emotion: {label} \n🔢 Raw Output: {pred}\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
📝 Input: I am very happy today! 
🔮 Predicted Emotion: joy 
🔢 Raw Output: [0.00835002 0.07188448 0.5765297  0.25955978 0.07846018 0.00521597]

📝 Input: I feel so sad and lonely. 
🔮 Predicted Emotion: sadness 
🔢 Raw Output: [0.02014122 0.00208538 0.07085428 0.0111003  0.8931015  0.00271722]

📝 Input: That was a scary experience. 
🔮 Predicted Emotion: joy 
🔢 Raw Output: [0.01268039 0.40368217 0.52362907 0.04468686 0.00993078 0.00539085]

📝 Input: I feel so intimidated by his actions 
🔮 Predicted Emotion: fear 
🔢 Raw Output: [3.4265917e-02 9.5649606e-01 7.4476073e-03 6.7920645e-04 9.5861644e-05
 1.0152187e-03]



In [ ]:

import numpy as np

# Create test sentences
test_sentences = ["I am very happy today!", "I feel so sad and lonely.", "That was a scary experience.","i dont know hoe life will turn out"]

# Tokenize and pad them just like training data
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, maxlen=max_length,padding="pre")

# Get predictions
predictions = model.predict(test_padded)
adjusted_predictions = le.inverse_transform(np.argmax(predictions, axis=1))

# Apply to predictions
adjusted_predictions = np.array([adjust_softmax(pred, temperature=0.5) for pred in predictions])

# Get final predicted index
emotion_indices = np.argmax(adjusted_predictions, axis=1)
for text, pred, adj_pred,idx in zip(test_sentences, predictions, adjusted_predictions,emotion_indices):
    print(f"📝 Input: {text} \n🔮 Raw Output: {pred} \n🔥 Adjusted Output: {adj_pred} \n {idx}\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
📝 Input: I am very happy today! 
🔮 Raw Output: [0.00835002 0.07188448 0.5765297  0.25955978 0.07846018 0.00521597] 
🔥 Adjusted Output: [1.6956859e-04 1.2567251e-02 8.0837601e-01 1.6384940e-01 1.4971612e-02
 6.6166962e-05] 
 2

📝 Input: I feel so sad and lonely. 
🔮 Raw Output: [0.02014122 0.00208538 0.07085428 0.0111003  0.8931015  0.00271722] 
🔥 Adjusted Output: [5.0507137e-04 5.4143861e-06 6.2504793e-03 1.5340891e-04 9.9307644e-01
 9.1924285e-06] 
 4

📝 Input: That was a scary experience. 
🔮 Raw Output: [0.01268039 0.40368217 0.52362907 0.04468686 0.00993078 0.00539085] 
🔥 Adjusted Output: [3.6590931e-04 3.7084067e-01 6.2395859e-01 4.5443093e-03 2.2442671e-04
 6.6133776e-05] 
 2

📝 Input: i dont know hoe life will turn out 
🔮 Raw Output: [0.19092481 0.36649638 0.15019567 0.10072362 0.14844911 0.04321039] 
🔥 Adjusted Output: [0.1603143  0.5907271  0.09921157 0.04461801 0.0969176  0.00821152] 
 1



In [ ]:
]
